In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("train_BRCpofr.csv")

In [3]:
df_n = pd.get_dummies(df)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89392 entries, 0 to 89391
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              89392 non-null  int64 
 1   gender          89392 non-null  object
 2   area            89392 non-null  object
 3   qualification   89392 non-null  object
 4   income          89392 non-null  object
 5   marital_status  89392 non-null  int64 
 6   vintage         89392 non-null  int64 
 7   claim_amount    89392 non-null  int64 
 8   num_policies    89392 non-null  object
 9   policy          89392 non-null  object
 10  type_of_policy  89392 non-null  object
 11  cltv            89392 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 8.2+ MB


In [5]:
df_n

,id,marital_status,vintage,claim_amount,cltv,gender_Female,gender_Male,area_Rural,area_Urban,qualification_Bachelor,...,income_<=2L,income_More than 10L,num_policies_1,num_policies_More than 1,policy_A,policy_B,policy_C,type_of_policy_Gold,type_of_policy_Platinum,type_of_policy_Silver
0,1,1,5,5790,64308,0,1,0,1,1,...,0,0,0,1,1,0,0,0,1,0
1,2,0,8,5080,515400,0,1,1,0,0,...,0,0,0,1,1,0,0,0,1,0
2,3,1,8,2599,64212,0,1,0,1,1,...,0,0,0,1,1,0,0,0,1,0
3,4,0,7,0,97920,1,0,1,0,0,...,0,0,0,1,1,0,0,0,1,0
4,5,1,6,3508,59736,0,1,0,1,0,...,0,1,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89387,89388,0,6,0,95736,0,1,1,0,0,...,0,1,0,1,0,1,0,0,1,0
89388,89389,1,0,3897,67596,0,1,0,1,1,...,0,0,0,1,1,0,0,0,1,0
89389,89390,0,4,3363,114120,1,0,0,1,1,...,0,0,0,1,0,1,0,0,1,0
89390,89391,0,8,0,71772,1,0,1,0,0,...,0,0,0,1,0,1,0,0,1,0


In [6]:
outliers = []
def detect_outliers_iqr(data):
    data = sorted(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    # print(q1, q3)
    IQR = q3-q1
    lwr_bound = q1-(1.5*IQR)
    upr_bound = q3+(1.5*IQR)
    # print(lwr_bound, upr_bound)
    for i in data: 
        if (i<lwr_bound or i>upr_bound):
            outliers.append(i)
    return outliers# Driver code
sample_outliers = detect_outliers_iqr(df_n['claim_amount'])
print("Outliers from IQR method: ", len(sample_outliers))

Outliers from IQR method:  2258


In [7]:
claim_Amount = df_n['claim_amount']

In [8]:
# Computing 10th, 90th percentiles and replacing the outliers
tenth_percentile = np.percentile(df_n['claim_amount'], 10)
ninetieth_percentile = np.percentile(df_n['claim_amount'], 90)
# print(tenth_percentile, ninetieth_percentile)
b = np.where(df_n['claim_amount']<tenth_percentile, tenth_percentile, df_n['claim_amount'])
b = np.where(b>ninetieth_percentile, ninetieth_percentile, b)
# print("Sample:", sample)
print("New array:",b)

New array: [5790. 5080. 2599. ... 3363.    0.    0.]


In [9]:
df_n.columns

Index(['id', 'marital_status', 'vintage', 'claim_amount', 'cltv',
       'gender_Female', 'gender_Male', 'area_Rural', 'area_Urban',
       'qualification_Bachelor', 'qualification_High School',
       'qualification_Others', 'income_2L-5L', 'income_5L-10L', 'income_<=2L',
       'income_More than 10L', 'num_policies_1', 'num_policies_More than 1',
       'policy_A', 'policy_B', 'policy_C', 'type_of_policy_Gold',
       'type_of_policy_Platinum', 'type_of_policy_Silver'],
      dtype='object')

In [10]:
df_n['claim_amount'] = b

In [11]:
df_n.rename(columns = {'income_<=2L':'income_less_than_2L'}, inplace = True)
df_n.rename(columns = {'income_2L-5L':'income_between_2L_to_5L'}, inplace = True)
df_n.rename(columns = {'income_5L-10L':'income_between_5L_to_10L'}, inplace = True)

In [12]:
df_n.drop(['id'], axis=1,inplace=True)

In [13]:
X=df_n.drop(['cltv'], axis=1)
y=df_n['cltv']

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 43)

In [15]:
from sklearn.ensemble import GradientBoostingRegressor

In [25]:
from sklearn.model_selection import GridSearchCV
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=3000,min_samples_leaf=100,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10), 
param_grid = param_test1, scoring='r2',n_jobs=4, cv=10)
gsearch1.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=GradientBoostingRegressor(max_depth=8,
                                                 max_features='sqrt',
                                                 min_samples_leaf=100,
                                                 min_samples_split=3000,
                                                 random_state=10,
                                                 subsample=0.8),
             n_jobs=4, param_grid={'n_estimators': range(20, 81, 10)},
             scoring='r2')

In [26]:
from sklearn.metrics import r2_score
y_pred = gsearch1.predict(X_test)
score = r2_score(y_test,y_pred)
print('r2 socre is ',score)

r2 socre is  0.15997328242574416


In [27]:
df_test = pd.read_csv("test_koRSKBP.csv")
df_test = pd.get_dummies(df_test)
df_test.rename(columns = {'income_<=2L':'income_less_than_2L'}, inplace = True)
df_test.rename(columns = {'income_2L-5L':'income_between_2L_to_5L'}, inplace = True)
df_test.rename(columns = {'income_5L-10L':'income_between_5L_to_10L'}, inplace = True)
df_test.drop(['id'], axis=1,inplace=True)

In [28]:
df_id = pd.read_csv("test_koRSKBP.csv")

In [30]:
y_actual_pred = gsearch1.predict(df_test)
df_y_pred = pd.DataFrame(y_actual_pred,columns=['cltv'])
df_lasso = pd.concat([df_id['id'], df_y_pred],axis=1)
df_lasso.to_csv('GBM.csv',index=False)

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xg
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [19]:
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
#dtc =  Ridge()
#dtc = LinearRegression()

gbm =  GradientBoostingRegressor()
xgb = XGBRegressor()
rf = RandomForestRegressor()
lgb = LGBMRegressor()
cat = CatBoostRegressor()

#clf = [('dtc',dtc),('rfc',rfc),('knn',knn),('DT',DT),('xgb',xgb)] #list of (str, estimator)
clf = [('rf',rf),('xgb',xgb),('lgb',lgb),('cat',cat)]

In [20]:
stack_model = StackingRegressor(estimators=clf, final_estimator=gbm)
#score = cross_val_score(stack_model,X_train,y_train,cv = 5,scoring = 'accuracy')
stack_model.fit(X_train,y_train)
y_pred = stack_model.predict(X_test)
print('-'*30)
r2 = r2_score(y_test,y_pred)
print('R2: ', r2)

Learning rate set to 0.078706
0:	learn: 89499.2742228	total: 140ms	remaining: 2m 19s
1:	learn: 88588.2053315	total: 149ms	remaining: 1m 14s
2:	learn: 87806.2093330	total: 159ms	remaining: 52.9s
3:	learn: 87131.7472236	total: 168ms	remaining: 41.9s
4:	learn: 86549.1957519	total: 176ms	remaining: 35s
5:	learn: 86039.8215960	total: 187ms	remaining: 31s
6:	learn: 85604.2214668	total: 196ms	remaining: 27.8s
7:	learn: 85240.7390912	total: 203ms	remaining: 25.2s
8:	learn: 84945.9982831	total: 209ms	remaining: 23s
9:	learn: 84662.2199083	total: 216ms	remaining: 21.4s
10:	learn: 84421.1820280	total: 223ms	remaining: 20.1s
11:	learn: 84218.0445921	total: 231ms	remaining: 19s
12:	learn: 84034.4639922	total: 238ms	remaining: 18.1s
13:	learn: 83878.3878253	total: 245ms	remaining: 17.3s
14:	learn: 83747.4766743	total: 252ms	remaining: 16.6s
15:	learn: 83631.4628755	total: 259ms	remaining: 15.9s
16:	learn: 83536.3432760	total: 266ms	remaining: 15.4s
17:	learn: 83450.8094072	total: 273ms	remaining: 14

162:	learn: 82243.7649243	total: 1.44s	remaining: 7.38s
163:	learn: 82239.6732171	total: 1.44s	remaining: 7.36s
164:	learn: 82235.0431454	total: 1.45s	remaining: 7.36s
165:	learn: 82229.9998595	total: 1.46s	remaining: 7.36s
166:	learn: 82227.3258575	total: 1.47s	remaining: 7.36s
167:	learn: 82223.7707589	total: 1.48s	remaining: 7.35s
168:	learn: 82219.2787152	total: 1.49s	remaining: 7.34s
169:	learn: 82212.1854954	total: 1.5s	remaining: 7.33s
170:	learn: 82208.6792322	total: 1.51s	remaining: 7.33s
171:	learn: 82202.8367264	total: 1.52s	remaining: 7.32s
172:	learn: 82198.8816709	total: 1.53s	remaining: 7.33s
173:	learn: 82196.7514534	total: 1.54s	remaining: 7.33s
174:	learn: 82191.7040298	total: 1.56s	remaining: 7.34s
175:	learn: 82187.6567689	total: 1.57s	remaining: 7.34s
176:	learn: 82183.3026856	total: 1.58s	remaining: 7.34s
177:	learn: 82177.8609011	total: 1.59s	remaining: 7.35s
178:	learn: 82172.2394794	total: 1.6s	remaining: 7.35s
179:	learn: 82169.2117438	total: 1.61s	remaining: 

319:	learn: 81650.7936293	total: 2.95s	remaining: 6.27s
320:	learn: 81647.3871032	total: 2.96s	remaining: 6.27s
321:	learn: 81646.2421738	total: 2.98s	remaining: 6.27s
322:	learn: 81643.9992970	total: 2.98s	remaining: 6.26s
323:	learn: 81641.5566531	total: 2.99s	remaining: 6.25s
324:	learn: 81637.5624105	total: 3s	remaining: 6.24s
325:	learn: 81635.1143535	total: 3.01s	remaining: 6.23s
326:	learn: 81631.5061538	total: 3.02s	remaining: 6.22s
327:	learn: 81629.6452553	total: 3.03s	remaining: 6.21s
328:	learn: 81625.3609776	total: 3.04s	remaining: 6.2s
329:	learn: 81622.1347814	total: 3.04s	remaining: 6.18s
330:	learn: 81620.2177991	total: 3.05s	remaining: 6.17s
331:	learn: 81618.7762149	total: 3.06s	remaining: 6.16s
332:	learn: 81613.6113474	total: 3.07s	remaining: 6.15s
333:	learn: 81609.1722006	total: 3.08s	remaining: 6.14s
334:	learn: 81606.3021073	total: 3.08s	remaining: 6.13s
335:	learn: 81603.4141869	total: 3.09s	remaining: 6.11s
336:	learn: 81598.6891074	total: 3.1s	remaining: 6.1

468:	learn: 81187.1084792	total: 4.21s	remaining: 4.77s
469:	learn: 81184.2644950	total: 4.22s	remaining: 4.76s
470:	learn: 81182.7937986	total: 4.23s	remaining: 4.75s
471:	learn: 81177.9351224	total: 4.24s	remaining: 4.74s
472:	learn: 81173.6360147	total: 4.25s	remaining: 4.73s
473:	learn: 81170.1959779	total: 4.25s	remaining: 4.72s
474:	learn: 81164.5791262	total: 4.26s	remaining: 4.71s
475:	learn: 81161.6573846	total: 4.27s	remaining: 4.7s
476:	learn: 81157.7336919	total: 4.28s	remaining: 4.69s
477:	learn: 81154.9320688	total: 4.29s	remaining: 4.68s
478:	learn: 81150.0372186	total: 4.3s	remaining: 4.67s
479:	learn: 81147.7639521	total: 4.3s	remaining: 4.66s
480:	learn: 81145.2995895	total: 4.31s	remaining: 4.65s
481:	learn: 81144.1133726	total: 4.32s	remaining: 4.64s
482:	learn: 81141.3113404	total: 4.33s	remaining: 4.63s
483:	learn: 81138.4751329	total: 4.33s	remaining: 4.62s
484:	learn: 81136.0899253	total: 4.34s	remaining: 4.61s
485:	learn: 81133.8784362	total: 4.35s	remaining: 4

634:	learn: 80689.2469928	total: 5.69s	remaining: 3.27s
635:	learn: 80687.2394771	total: 5.69s	remaining: 3.26s
636:	learn: 80684.7055283	total: 5.7s	remaining: 3.25s
637:	learn: 80682.7903346	total: 5.71s	remaining: 3.24s
638:	learn: 80680.7410170	total: 5.72s	remaining: 3.23s
639:	learn: 80678.2982371	total: 5.73s	remaining: 3.22s
640:	learn: 80675.6710475	total: 5.74s	remaining: 3.21s
641:	learn: 80673.3289521	total: 5.75s	remaining: 3.2s
642:	learn: 80667.5519602	total: 5.75s	remaining: 3.19s
643:	learn: 80665.5543063	total: 5.76s	remaining: 3.18s
644:	learn: 80663.4188959	total: 5.77s	remaining: 3.17s
645:	learn: 80660.7136703	total: 5.78s	remaining: 3.17s
646:	learn: 80657.8659032	total: 5.79s	remaining: 3.16s
647:	learn: 80655.9229248	total: 5.79s	remaining: 3.15s
648:	learn: 80653.4605526	total: 5.8s	remaining: 3.14s
649:	learn: 80648.9295058	total: 5.81s	remaining: 3.13s
650:	learn: 80646.7776957	total: 5.82s	remaining: 3.12s
651:	learn: 80644.1154663	total: 5.83s	remaining: 3

...

351:	learn: 81754.9579320	total: 2.5s	remaining: 4.59s
352:	learn: 81750.1780591	total: 2.5s	remaining: 4.59s
353:	learn: 81743.6255502	total: 2.51s	remaining: 4.58s
354:	learn: 81738.7310363	total: 2.52s	remaining: 4.58s
355:	learn: 81735.6993629	total: 2.53s	remaining: 4.57s
356:	learn: 81731.8549661	total: 2.53s	remaining: 4.56s
357:	learn: 81724.7820896	total: 2.54s	remaining: 4.55s
358:	learn: 81721.5588207	total: 2.55s	remaining: 4.55s
359:	learn: 81714.4851266	total: 2.55s	remaining: 4.54s
360:	learn: 81711.0931301	total: 2.56s	remaining: 4.53s
361:	learn: 81707.7122653	total: 2.56s	remaining: 4.52s
362:	learn: 81704.1203209	total: 2.57s	remaining: 4.51s
363:	learn: 81701.0948798	total: 2.58s	remaining: 4.51s
364:	learn: 81697.9733701	total: 2.59s	remaining: 4.5s
365:	learn: 81693.3334133	total: 2.59s	remaining: 4.49s
366:	learn: 81688.8798713	total: 2.6s	remaining: 4.48s
367:	learn: 81685.3001388	total: 2.6s	remaining: 4.47s
368:	learn: 81683.1058037	total: 2.61s	remaining: 4.4

517:	learn: 81118.0767127	total: 3.72s	remaining: 3.46s
518:	learn: 81115.3371938	total: 3.73s	remaining: 3.45s
519:	learn: 81111.1463239	total: 3.73s	remaining: 3.45s
520:	learn: 81108.6012221	total: 3.74s	remaining: 3.44s
521:	learn: 81103.6997899	total: 3.75s	remaining: 3.43s
522:	learn: 81101.6950711	total: 3.75s	remaining: 3.42s
523:	learn: 81096.8258795	total: 3.76s	remaining: 3.42s
524:	learn: 81093.8979703	total: 3.77s	remaining: 3.41s
525:	learn: 81089.8408326	total: 3.77s	remaining: 3.4s
526:	learn: 81086.6910081	total: 3.78s	remaining: 3.39s
527:	learn: 81084.4051630	total: 3.79s	remaining: 3.38s
528:	learn: 81079.4751609	total: 3.79s	remaining: 3.38s
529:	learn: 81074.8501268	total: 3.8s	remaining: 3.37s
530:	learn: 81072.6729700	total: 3.8s	remaining: 3.36s
531:	learn: 81069.8731505	total: 3.81s	remaining: 3.35s
532:	learn: 81065.4501056	total: 3.81s	remaining: 3.34s
533:	learn: 81062.6535230	total: 3.82s	remaining: 3.33s
534:	learn: 81060.6343821	total: 3.83s	remaining: 3

674:	learn: 80599.9985030	total: 4.79s	remaining: 2.31s
675:	learn: 80598.1752591	total: 4.8s	remaining: 2.3s
676:	learn: 80594.3876534	total: 4.81s	remaining: 2.29s
677:	learn: 80592.3229395	total: 4.82s	remaining: 2.29s
678:	learn: 80588.7819083	total: 4.82s	remaining: 2.28s
679:	learn: 80587.2208880	total: 4.83s	remaining: 2.27s
680:	learn: 80585.0884565	total: 4.84s	remaining: 2.27s
681:	learn: 80581.5043381	total: 4.84s	remaining: 2.26s
682:	learn: 80578.9152342	total: 4.85s	remaining: 2.25s
683:	learn: 80576.8330340	total: 4.86s	remaining: 2.25s
684:	learn: 80574.5526195	total: 4.87s	remaining: 2.24s
685:	learn: 80572.4007069	total: 4.87s	remaining: 2.23s
686:	learn: 80568.6598290	total: 4.88s	remaining: 2.22s
687:	learn: 80567.1712701	total: 4.89s	remaining: 2.22s
688:	learn: 80562.2574264	total: 4.89s	remaining: 2.21s
689:	learn: 80559.9599690	total: 4.9s	remaining: 2.2s
690:	learn: 80552.4310089	total: 4.91s	remaining: 2.19s
691:	learn: 80550.4705676	total: 4.92s	remaining: 2.

822:	learn: 80167.4540629	total: 5.88s	remaining: 1.26s
823:	learn: 80162.0525245	total: 5.88s	remaining: 1.26s
824:	learn: 80157.9283852	total: 5.89s	remaining: 1.25s
825:	learn: 80153.9875146	total: 5.91s	remaining: 1.24s
826:	learn: 80150.3479554	total: 5.91s	remaining: 1.24s
827:	learn: 80148.4568953	total: 5.92s	remaining: 1.23s
828:	learn: 80142.4144010	total: 5.93s	remaining: 1.22s
829:	learn: 80139.8071327	total: 5.93s	remaining: 1.22s
830:	learn: 80138.1751312	total: 5.94s	remaining: 1.21s
831:	learn: 80133.8193464	total: 5.95s	remaining: 1.2s
832:	learn: 80131.1277934	total: 5.96s	remaining: 1.19s
833:	learn: 80126.8093989	total: 5.97s	remaining: 1.19s
834:	learn: 80123.2640342	total: 5.97s	remaining: 1.18s
835:	learn: 80119.3913319	total: 5.98s	remaining: 1.17s
836:	learn: 80116.1534010	total: 5.99s	remaining: 1.17s
837:	learn: 80111.0632057	total: 6s	remaining: 1.16s
838:	learn: 80107.7896310	total: 6s	remaining: 1.15s
839:	learn: 80105.7743883	total: 6.01s	remaining: 1.14s

969:	learn: 79719.8712729	total: 6.97s	remaining: 215ms
970:	learn: 79715.4688343	total: 6.97s	remaining: 208ms
971:	learn: 79712.4106941	total: 6.98s	remaining: 201ms
972:	learn: 79709.0627610	total: 6.99s	remaining: 194ms
973:	learn: 79707.6478670	total: 7s	remaining: 187ms
974:	learn: 79704.4297386	total: 7s	remaining: 180ms
975:	learn: 79703.0500990	total: 7.01s	remaining: 172ms
976:	learn: 79699.7408163	total: 7.02s	remaining: 165ms
977:	learn: 79697.2575102	total: 7.02s	remaining: 158ms
978:	learn: 79693.4352100	total: 7.03s	remaining: 151ms
979:	learn: 79689.7426868	total: 7.04s	remaining: 144ms
980:	learn: 79687.5982660	total: 7.04s	remaining: 136ms
981:	learn: 79683.5914248	total: 7.05s	remaining: 129ms
982:	learn: 79680.1075013	total: 7.06s	remaining: 122ms
983:	learn: 79675.9465226	total: 7.07s	remaining: 115ms
984:	learn: 79672.9422654	total: 7.07s	remaining: 108ms
985:	learn: 79668.6509799	total: 7.08s	remaining: 101ms
986:	learn: 79667.2634106	total: 7.09s	remaining: 93.3